In [1]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

/tmp/ipykernel_1506242/2590453881.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import spacy
nlp = spacy.load('en_core_web_lg')

from spacy import displacy

In [3]:
%%capture

import sys
sys.path.append('../src/')

from model_ner import HUGFACE_NER
from AIONER_Run import ML_Tag 

2024-06-20 15:31:51.195502: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-20 15:31:51.210178: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 15:31:51.210192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 15:31:51.210879: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-20 15:31:51.214288: I tensorflow/core/platform/cpu_feature_guar

In [4]:
texts = pd.read_csv('../data/PubMed_Jun24/PubMed_Jun24.csv').sample(10).reset_index(drop=True)
texts['text'] = texts['title'] + ' ' + texts['abstract']

In [5]:
texts.head(1)

,pmid,title,abstract,text
0,38886960,Salsolinol as an RNA m6A methylation inducer m...,JOURNAL/nrgr/04.03/01300535-202503000-00032/fi...,Salsolinol as an RNA m6A methylation inducer m...


# Core entities

In [6]:
entities = ['Chemical', 'Gene', 'Disease', 'Species', 'Variant', 'CellLine']

In [7]:
COLORS = plt.cycler("color", plt.cm.Set3.colors)
COLORS = [matplotlib.colors.to_hex(c['color']) for c in COLORS]
COLORS = dict(zip(entities,COLORS[:len(entities)]))

## BiomedBERT-base abstract

In [8]:
vocabfiles={'labelfile':'../vocab/AIO_label.vocab',
            'checkpoint_path':'../pretrained_models/BiomedNLP-PubMedBERT-base-uncased-abstract/',
            'lowercase':True
           }

In [9]:
%%capture

nn_model = HUGFACE_NER(vocabfiles)
nn_model.build_encoder()

nn_model.build_crf_decoder()

nn_model.load_model('../models/pubmedbert/pubmedbert-es-AIO.h5')

2024-06-20 15:31:52.497141: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-20 15:31:52.497259: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-20 15:31:52.497317: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [10]:
for text in texts['text'].values:
    doc = nlp(text)
    
    spans = []
    for ent in ML_Tag(text, ml_model=nn_model, entity_type='ALL'):
        spans.append(doc.char_span(int(ent[0]), int(ent[1]), ent[2].upper()))
    
    try:
        doc.ents = spans
    except:
        doc.ents = []
    
    displacy.render(doc, style="ent", options={'colors': COLORS})

    print('*'*150)
    print('\n')
    print('*'*150)

1/1 [==============================] - 2s 2s/step


******************************************************************************************************************************************************


******************************************************************************************************************************************************
1/1 [==============================] - 0s 56ms/step


******************************************************************************************************************************************************


******************************************************************************************************************************************************
1/1 [==============================] - 0s 61ms/step


******************************************************************************************************************************************************


******************************************************************************************************************************************************
1/1 [==============================] - 0s 69ms/step


******************************************************************************************************************************************************


******************************************************************************************************************************************************
1/1 [==============================] - 0s 67ms/step


...

******************************************************************************************************************************************************


******************************************************************************************************************************************************
1/1 [==============================] - 0s 68ms/step


******************************************************************************************************************************************************


******************************************************************************************************************************************************
1/1 [==============================] - 0s 68ms/step


******************************************************************************************************************************************************


******************************************************************************************************************************************************
1/1 [==============================] - 0s 69ms/step


******************************************************************************************************************************************************


******************************************************************************************************************************************************
1/1 [==============================] - 0s 58ms/step


******************************************************************************************************************************************************


******************************************************************************************************************************************************


# AnatEM entities

In [11]:
entities = ['Cell_component', 'Tissue', 'Cancer', 'Organ', 'Multi-tissue_structure']

In [12]:
COLORS = plt.cycler("color", plt.cm.Set3.colors)
COLORS = [matplotlib.colors.to_hex(c['color']) for c in COLORS]
COLORS = dict(zip(entities,COLORS[:len(entities)]))

## BiomedBERT-base abstract

In [13]:
vocabfiles={'labelfile':'../vocab/AnatEM_selected_label.vocab',
            'checkpoint_path':'../pretrained_models/BiomedNLP-PubMedBERT-base-uncased-abstract/',
            'lowercase':True
           }

In [14]:
%%capture

nn_model = HUGFACE_NER(vocabfiles)
nn_model.build_encoder()

nn_model.build_crf_decoder()

nn_model.load_model('../models/AnatEM/pubmedbert/pubmedbert-best-finetune.h5')

In [15]:
for text in texts['text'].values:
    doc = nlp(text)
    
    spans = []
    for ent in ML_Tag(text, ml_model=nn_model, entity_type='ALL'):
        spans.append(doc.char_span(int(ent[0]), int(ent[1]), ent[2].upper()))
    
    try:
        doc.ents = spans
    except:
        doc.ents = []
    
    displacy.render(doc, style="ent", options={'colors': COLORS})

    print('*'*150)
    print('\n')
    print('*'*150)

1/1 [==============================] - 1s 1s/step


******************************************************************************************************************************************************


******************************************************************************************************************************************************
1/1 [==============================] - 0s 58ms/step


******************************************************************************************************************************************************


******************************************************************************************************************************************************
1/1 [==============================] - 0s 57ms/step


******************************************************************************************************************************************************


******************************************************************************************************************************************************
1/1 [==============================] - 0s 66ms/step


******************************************************************************************************************************************************


******************************************************************************************************************************************************
1/1 [==============================] - 0s 77ms/step


...

******************************************************************************************************************************************************


******************************************************************************************************************************************************
1/1 [==============================] - 0s 70ms/step


******************************************************************************************************************************************************


******************************************************************************************************************************************************
1/1 [==============================] - 0s 75ms/step


******************************************************************************************************************************************************


******************************************************************************************************************************************************
1/1 [==============================] - 0s 61ms/step


******************************************************************************************************************************************************


******************************************************************************************************************************************************
1/1 [==============================] - 0s 63ms/step


******************************************************************************************************************************************************


******************************************************************************************************************************************************
